In [ ]:
# !pip install rasterio PyCRS geedim


In [ ]:
import os
import numpy as np
import rasterio
from rasterio import features
from shapely.geometry import shape, mapping
import fiona

In [ ]:
def process_tiff(input_tiff, output_shapefile):
    with rasterio.open(input_tiff) as f:
        image = f.read(4)  # Alpha band
        nodata = 0  # Consider where alpha is 0 as nodata (use f.nodata if available)

        # Mask: 0 where there's nodata, 1 where it's valid
        is_valid = (image != nodata).astype(np.uint8)

        # Schema for the shapefile
        schema = {
            'geometry': 'Polygon',
            'properties': {'id': 'int'},
        }

        with fiona.open(output_shapefile, 'w', 'ESRI Shapefile', schema, crs=f.crs) as output:
            # Vectorize the binary image, supplying the transform so it returns mapped coords
            for geom, value in features.shapes(is_valid, transform=f.transform):
                # Ignore nodata
                if value != 0:
                    shapely_geom = shape(geom) # Convert geojson to shapely geometry

                    output.write({
                        'geometry': mapping(shapely_geom),
                        'properties': {'id': 1},
                    })


def process_folder(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".tif") or file.lower().endswith(".tiff"):
                try:

                  tiff_path = os.path.join(root, file)
                  output_shapefile = os.path.join(output_folder, os.path.splitext(file)[0] + "_shapefile.shp")
                  process_tiff(tiff_path, output_shapefile)
                except:
                  print(f'{file} Failed')
                  continue

In [ ]:
input_folder = "path/to/imagery"
output_folder = "destination/path/of/shapefiles"

os.makedirs(output_folder, exist_ok=True)

process_folder(input_folder, output_folder)